![imagen](img/foto1.png)

![imagen](img/foto2.png)

Para ejercitarte y afianzar lo aprendido sobre **Embeddings y procesamiento de texto**, completa los siguientes ejercicios. Recuerda que necesitarás datos que están en el directorio data que acompaña al notebook (búscalo en el repositorio de ejercicios).

La solución a los mismos las tienes ya, y en este caso se te invita a QUE SIGAS EL EJERCICIO CON LA SOLUCION a modo de tutorial ya que hay varios aspectos que son nuevos y se introducen en la solución.

### Ejercicio 0

Importa las librerías necesarias

In [1]:
import io
import os
import re
import shutil
import string

import numpy as np
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Dense, Embedding, GlobalAveragePooling1D,
                                     TextVectorization)

### Ejercicio 1: Descarga el dataset

Usarás el [Conjunto de Datos de Grandes Reseñas de Películas](http://ai.stanford.edu/~amaas/data/sentiment/) a lo largo del tutorial. Entrenarás un modelo de clasificador de sentimientos con este conjunto de datos y, en el proceso, aprenderás embeddings desde cero. 

Descarga el conjunto de datos utilizando la utilidad de archivos de Keras y echa un vistazo a los directorios.

In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz", 
                                  origin=url,
                                  extract=True, 
                                  cache_dir='./data/',
                                  cache_subdir='./')

84125825/84125825 ━━━━━━━━━━━━━━━━━━━━ 69s 1us/step


In [3]:
print(dataset)

./data/./aclImdb_v1_extracted


In [4]:
print(os.getcwd())

c:\Users\Lander\Documents\GitHub\ONLINE_DS_THEBRIDGE_2024\06_Deep_Learning\S20_Redes_Recurrentes_y_Embeddings\U02_Capas_Prepocesado_y_Embedings\Ejercicios_Workout


### Ejercicio 2

Echa un vistazo al directorio train/. Tiene carpetas pos y neg con reseñas de películas etiquetadas como positivas y negativas respectivamente. Utilizarás reseñas de las carpetas pos y neg para entrenar un modelo de clasificación binaria.

In [5]:
dataset_dir = './data/aclImdb_v1_extracted/aclImdb'
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['labeledBow.feat',
 'neg',
 'pos',
 'unsup',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

### Ejercicio 3

El directorio `train` también tiene carpetas adicionales que deberían ser eliminadas antes de crear el conjunto de datos de entrenamiento.

In [6]:
# delete path
shutil.rmtree(dataset_dir + "/train/unsup")

### Ejercicio 4

A continuación, crea un tf.data.Dataset usando tf.keras.preprocessing.text_dataset_from_directory. Puedes leer más sobre cómo utilizar esta utilidad en este [tutorial de clasificación de texto](https://www.tensorflow.org/tutorials/keras/text_classification).

Usa el directorio de entrenamiento para crear conjuntos de datos de entrenamiento y validación con una división del 20% para la validación.

In [7]:
batch_size = 1024
seed = 123

'''
Busca en un directorio todas las carpetas. Cada carpeta es una etiqueta
Y cada archivo una review. Podemos especificar si es para el subset
de training o validation y cuanto dejamos para validacion.
Esto crea un tf.data.Dataset
ELIMINAR UNA CARPETA EN TRAIN, QUE SOBRA
'''

train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation', # Esto y la semilla permiten que las muestras con train no se superpongan
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [ ]:
# WARNING: no se si esta forma también sería correcta

# batch_size = 1024
# seed = 123

# '''
# Busca en un directorio todas las carpetas. Cada carpeta es una etiqueta
# Y cada archivo una review. Podemos especificar si es para el subset
# de training o validation y cuanto dejamos para validacion.
# Esto crea un tf.data.Dataset
# ELIMINAR UNA CARPETA EN TRAIN, QUE SOBRA
# '''

# train_ds, val_ds = tf.keras.preprocessing.text_dataset_from_directory(
#     train_dir,
#     batch_size=batch_size,
#     validation_split=0.2,
#     subset='both',
#     seed=seed)

*Echa un vistazo a algunas reseñas de películas y sus etiquetas (1: positiva, 0: negativa) del conjunto de datos de entrenamiento.*


In [9]:
# Cogemos el primer batch
for text_batch, label_batch in train_ds.take(1): 
    for i in range(5): # 1025 da error xq no hay mas en este batch
        print(text_batch.numpy()[i][:150], "...")
        print("Label:", label_batch[i].numpy(), "(%s)" %("Positive" if label_batch.numpy()[i] == 1 else "Negative"))

b'Oh My God! Please, for the love of all that is holy, Do Not Watch This Movie! It it 82 minutes of my life I will never get back. Sure, I could have st' ...
Label: 0 (Negative)
b'This movie is SOOOO funny!!! The acting is WONDERFUL, the Ramones are sexy, the jokes are subtle, and the plot is just what every high schooler dreams' ...
Label: 1 (Positive)
b'Alex D. Linz replaces Macaulay Culkin as the central figure in the third movie in the Home Alone empire. Four industrial spies acquire a missile guida' ...
Label: 0 (Negative)
b"There's a good movie lurking here, but this isn't it. The basic idea is good: to explore the moral issues that would face a group of young survivors o" ...
Label: 0 (Negative)
b'I saw this movie at an actual movie theater (probably the $2.00 one) with my cousin and uncle. We were around 11 and 12, I guess, and really into scar' ...
Label: 0 (Negative)


### Ejercicio 5: Configura el dataset para mejorar el rendimiento (mira la solución)

Estos son dos métodos importantes que deberías usar al cargar datos para asegurarte de que las operaciones de entrada/salida no se conviertan en un bloqueo.

`.cache()` mantiene los datos en memoria después de ser cargados desde el disco. Esto garantizará que el conjunto de datos no se convierta en un cuello de botella mientras entrenas tu modelo. Si tu conjunto de datos es demasiado grande para caber en la memoria, también puedes usar este método para crear una caché en disco eficiente, la cual es más eficiente para leer que muchos archivos pequeños.

`.prefetch()` solapa el preprocesado de datos y la ejecución del modelo durante el entrenamiento.

Puedes aprender más sobre ambos métodos, así como cómo cachear datos en disco en la [guía de rendimiento de datos](https://www.tensorflow.org/guide/data_performance).



In [10]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Ejercicio 6: Repasando Embeddings

Crea una capa de Embedding con dimensión de entrada 1000 y dimensión de salida 4. Codifica las frases "Me llamo Iñigo Montoya", "Tú mataste a mi padre", "Disponte a morir" (tendrás que crear una capa adicional). Haz la codificación de cada una por separado y luego prueba a ponerlas todas juntas en una misma lista. Ojo, cada frase la tienes que convertir a una lista de palabras. ¿Qué ocurre en este último caso? [Haz el ejercicio y luego mira la solución]

In [11]:
frases = ["Me llamo Iñigo Montoya", "Tú mataste a mi padre", "Disponte a morir"]
print(" ".join(frases).split())

['Me', 'llamo', 'Iñigo', 'Montoya', 'Tú', 'mataste', 'a', 'mi', 'padre', 'Disponte', 'a', 'morir']


In [ ]:
frases = ["Me llamo Iñigo Montoya", "Tú mataste a mi padre", "Disponte a morir"]

layer_test = Embedding(input_dim=1000, output_dim=4)
pre_procesado = tf.keras.layers.StringLookup()
pre_procesado.adapt(" ".join(frases).split())

conversor_fake = tf.keras.models.Sequential([pre_procesado,
                                             layer_test])

resultados = []
for frase in frases:
    resultados.append(conversor_fake(tf.constant([frase.split()])))
    print(f"Para <{frase}>, embeddings:\n{resultados[-1].numpy()}")

Para <Me llamo Iñigo Montoya>, embeddings:
[[[ 0.04838357 -0.02706586 -0.01583712 -0.01494152]
  [ 0.03206224  0.03040278  0.01336911  0.03139034]
  [ 0.01332668  0.04663992  0.01068345 -0.01346766]
  [ 0.04013517 -0.01368649  0.02875055 -0.04734081]]]
Para <Tú mataste a mi padre>, embeddings:
[[[-0.04695284 -0.04246197 -0.03199855  0.0075048 ]
  [-0.0225799  -0.03721614 -0.00847229 -0.00089647]
  [ 0.0371753   0.00765828  0.02573992  0.01082981]
  [-0.04406366 -0.01233744  0.00857102  0.01664624]
  [-0.04881733 -0.01097306 -0.00337977  0.0275001 ]]]
Para <Disponte a morir>, embeddings:
[[[-0.01865578 -0.02029446  0.02361711 -0.02638083]
  [ 0.0371753   0.00765828  0.02573992  0.01082981]
  [ 0.02794869 -0.04948401  0.00572672 -0.03148983]]]


*Para problemas de texto o secuencias, la capa Embedding toma un tensor 2D de enteros, de forma (muestras, longitud_de_secuencia), donde cada entrada es una secuencia de enteros. Puede incrustar secuencias de longitudes variables. Podrías alimentar la capa de incrustación con lotes de formas (32, 10) (lote de 32 secuencias de longitud 10) o (64, 15) (lote de 64 secuencias de longitud 15). Como se ve al meter frasess de diferente tamaño POR SEPARADO, pero fijate en cuando pruebas con todas las frases juntas.*

*Por otro lado El tensor devuelto tiene un eje más que la entrada, los vectores de incrustación se alinean a lo largo del nuevo último eje. Pásale un lote de entrada (2, 3) y la salida es (2, 3, N). Es decir, conjuntando lo anterior y este punto, si le pasas secuencias de diferente tamaño te devuelve secuencias de diferente tamaño y eso tendremos que "ajustarlo" para las siguientes capas. Ahora hagamos el intento de lanzar todo junto:*

In [13]:
conversor_fake(tf.constant([frase.split() for frase in frases]))

ValueError: Can't convert non-rectangular Python sequence to Tensor.

*Los tensores son estructuras "rectangulares" eso de meterle secuencias de longitud variable nos lleva a un primer punto que debéis conocer si vais a trabajar con NLP en DL... __Es necesario hacer Padding__, es decir hacer secuencias de un tamaño fijo rellenando con "ceros" las secuencias más cortas.*



In [14]:
tam_maximo = max([len(frase.split()) for frase in frases])
print(tam_maximo, end='\n\n')

frases_padded = []
for frase in frases:
    sentence = frase.split()
    print(sentence, len(sentence), tam_maximo)
    sentence += ["<relleno>" for i in range(len(sentence),tam_maximo)]
    print(sentence)
    frases_padded.append(sentence)

print('\nFrases padded:', frases_padded, sep='\n')
print(tf.constant(frases_padded))
conversor_fake(tf.constant(frases_padded))

5

['Me', 'llamo', 'Iñigo', 'Montoya'] 4 5
['Me', 'llamo', 'Iñigo', 'Montoya', '<relleno>']
['Tú', 'mataste', 'a', 'mi', 'padre'] 5 5
['Tú', 'mataste', 'a', 'mi', 'padre']
['Disponte', 'a', 'morir'] 3 5
['Disponte', 'a', 'morir', '<relleno>', '<relleno>']

Frases padded:
[['Me', 'llamo', 'Iñigo', 'Montoya', '<relleno>'], ['Tú', 'mataste', 'a', 'mi', 'padre'], ['Disponte', 'a', 'morir', '<relleno>', '<relleno>']]
tf.Tensor(
[[b'Me' b'llamo' b'I\xc3\xb1igo' b'Montoya' b'<relleno>']
 [b'T\xc3\xba' b'mataste' b'a' b'mi' b'padre']
 [b'Disponte' b'a' b'morir' b'<relleno>' b'<relleno>']], shape=(3, 5), dtype=string)


<tf.Tensor: shape=(3, 5, 4), dtype=float32, numpy=
array([[[ 0.04838357, -0.02706586, -0.01583712, -0.01494152],
        [ 0.03206224,  0.03040278,  0.01336911,  0.03139034],
        [ 0.01332668,  0.04663992,  0.01068345, -0.01346766],
        [ 0.04013517, -0.01368649,  0.02875055, -0.04734081],
        [-0.04411323, -0.02603287,  0.00755753,  0.04467168]],

       [[-0.04695284, -0.04246197, -0.03199855,  0.0075048 ],
        [-0.0225799 , -0.03721614, -0.00847229, -0.00089647],
        [ 0.0371753 ,  0.00765828,  0.02573992,  0.01082981],
        [-0.04406366, -0.01233744,  0.00857102,  0.01664624],
        [-0.04881733, -0.01097306, -0.00337977,  0.0275001 ]],

       [[-0.01865578, -0.02029446,  0.02361711, -0.02638083],
        [ 0.0371753 ,  0.00765828,  0.02573992,  0.01082981],
        [ 0.02794869, -0.04948401,  0.00572672, -0.03148983],
        [-0.04411323, -0.02603287,  0.00755753,  0.04467168],
        [-0.04411323, -0.02603287,  0.00755753,  0.04467168]]],
      dtype=f

*Esto resuelve dos cosas: el hecho de que podamos meter las frases como batches completos, y la salida que también está fija, pero traerá otro problema, la descompensación entre frases largas y cortas, ya que en estas últimas habrá mucho "relleno". La forma de solucionarlo se llama "masking" o enmascaramiento y se hará usando una máscara por secuencia para indicarle a las capas qué valores son de relleno y no se han de usar*

*Además del padding, para convertir de esta secuencia de longitud variable a una representación fija, hay una variedad de enfoques estándar. Podrías usar una capa RNN, Atención, o de agrupación antes de pasarla a una capa Densa. En estos ejercicios vamos a usar la agrupación con una capa de Pooling.*

### Ejercicio 7: Procesado de Texto con vectorización

A continuación, define los pasos de preprocesamiento del conjunto de datos necesarios para tu modelo de clasificación de sentimientos. Inicializa una capa de TextVectorization con los parámetros deseados para vectorizar las reseñas de películas. Recuerda que tendrás que limipiar (tambien se llama "estandandizar") las reseñas (como lo hicimos en su día o como en el workout o como lo hacemos, diferente, en la solución).

Importante: En este ejercicio vamos a usar la capa de vectorización pero no para convertir las frases en conteos de palabras o sus tfidf, sino en listas de indices en un vocabulario. Por eso a la hora de inicializar la capa debes usar el output_mode = "int" (en el workout lo hemos empleado con "count" y con "tf-idf". Configura la capa de forma que ajuste su salida a secuencias o frases de 8 palabras.

Prueba la nueva capa con las frases del ejercicio anterior.


In [15]:
# Creamos una función de limpieza
def custom_cleaner(input_data):
    """
    Función de limpieza de datos.
    Convierte a minúsculas, quita los codigos de cambio de línea, y quita los signos de puntuación
    """
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                    '[%s]' % re.escape(string.punctuation), 
                                    '')


sequence_length = 8

# Veamos como la capa TextVectorization nos permite darle secuenccias de entrada y
# nos devuelve el formato lista de índices, CON PADDING
# el modo int eso hace que devuelva una secuencia con índices al vocabulario IMPORTANTE!!!
# output_sequence_length --> indica el tamaño de la secuencia de salida y con eso ya estamos diciéndole como queremos hacer el Padding.
vectorize_layer = TextVectorization(
    standardize=custom_cleaner,
    output_mode='int', 
    output_sequence_length=sequence_length
    ) 


In [16]:
# Fit de las frases
vectorize_layer.adapt(frases)
vectorize_layer(frases)

<tf.Tensor: shape=(3, 8), dtype=int64, numpy=
array([[ 8, 10, 11,  6,  0,  0,  0,  0],
       [ 3,  9,  2,  7,  4,  0,  0,  0],
       [12,  2,  5,  0,  0,  0,  0,  0]], dtype=int64)>

*Fijate como ha hecho el padding hasta llegar a los 8 terminos que le hemos dado como sequence_length*

In [17]:
for indice, word in enumerate(vectorize_layer.get_vocabulary()):
    print("%d --> %s" %(indice,word))

0 --> 
1 --> [UNK]
2 --> a
3 --> tú
4 --> padre
5 --> morir
6 --> montoya
7 --> mi
8 --> me
9 --> mataste
10 --> llamo
11 --> iñigo
12 --> disponte


El índice 0 se suele reservar para hacer el padding, como puedes observar en el resultado al aplicar la capa a las frases del ejercicio anterior.

Ahora a la salida del TextVectorization le aplicaremos la capa de Embeddings y luego a eso nuestras capas densas para hacer el modelo de clasificación

### Ejercicio 8

Recrea la capa del ejercicio anterior, llamándola vectorizer_layer. pero esta vez para un vocabulario de 1000 términos y para un tamaño de secuencia de 100 palabras. LUEGO ejecuta el código que tienes en la siguiente celda.

In [ ]:

def custom_standardization(input_data):
    """
    Create a custom standardization function to strip HTML break tags '<br />'.
    Lo convierte a minúsculas, le quita los codigos de cambio de línea, y le quita los signos de puntuación
    """
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ') 
    return tf.strings.regex_replace(stripped_html,
                                    '[%s]' % re.escape(string.punctuation), 
                                    '')


# Vocabulary size and number of words in a sequence.
# Vamos a permitir que tenga un vocabulario de 10000, las de mayor frecuencia 
# el resto se codificará como UNK, o si hemos empleado OOV como esas categorías extra
vocab_size = 10000 
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)


**Para ejecutar después de la creación de la capa vectorize_layer:**

In [19]:
# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
# Nos quitamos los labels de esta manera, que van en conjunto con features en train_ds
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

### Ejercicio 9

Es hora de crear el modelo de clasificación. Tendrás que emplear una capa ["GlobalAveragePooling1D"](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalAveragePooling1D) e investigar por tu cuenta un poco sobre ella, aunque aquí te dejo algunos apuntes. En definitiva, tu modelo tiene que tener:

* La capa [`TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization) transforma cadenas en índices de vocabulario. Ya has inicializado `vectorize_layer` como una capa de TextVectorization y has construido su vocabulario llamando a `adapt` en `text_ds`. Ahora, vectorize_layer puede ser utilizada como la primera capa de tu modelo de clasificación de principio a fin, alimentando cadenas transformadas en la capa de Embedding. Utiliza una dimensión de salida de 16 en el embedding.
  
* La capa [`Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) toma el vocabulario codificado en enteros y busca el vector de incrustación para cada índice de palabra. Estos vectores se aprenden a medida que el modelo se entrena. Los vectores añaden una dimensión al arreglo de salida. Las dimensiones resultantes son: `(lote, secuencia, incrustación)`.

* La capa [`GlobalAveragePooling1D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalAveragePooling1D) devuelve un vector de salida de longitud fija para cada ejemplo promediando sobre la dimensión de secuencia. Esto permite al modelo manejar entrada de longitud variable, de la manera más simple posible. Esta capa hace el sentence embedding que comentamos en el workout, es decir convierte la sentencia en un embedding que es el resultado de hacer la media de cada uno de sus word embeddings (sí es el centroide de sus embeddings) ya que es la forma más sencilla de hacer sentence embedding.

* El vector de salida de longitud fija se pasa a través de una capa completamente conectada ([`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)) con 16 unidades ocultas. (porque tenemos 16 de dimensión del embedding de palabra y de sentencia).

* La última capa está densamente conectada con un único nodo de salida (por ser un clasificador binario).

Precaución: Este modelo no utiliza enmascaramiento, por lo que el relleno de ceros se utiliza como parte de la entrada y, por lo tanto, la longitud del relleno puede afectar la salida. Para solucionar esto, consulta la [guía de enmascaramiento y relleno](https://www.tensorflow.org/guide/keras/masking_and_padding). Básicamente, al hacer el sentence embedding en frases cortas (imaginate una de una sola palabra, tendrá en nuestro caso 1 word-embedding y 99 de relleno) el embedding de la sentencia se ve completamente sesgado hacia el de relleno, por eso se usa el masking para decirle a la capa que sólo tenga en cuenta el embedding que no sea de relleno.y relleno.

In [ ]:
vocab_size = 10000 
embedding_dim = 16

'''
GlobalAveragePooling1D
Cada palabra tiene asociado un embedding. El ouput es la media de cada
coordenada del embedding, por tanto, si hay 16 embeddings, hará un
flatten a 16, siendo cada valor la media de la coordenada de ese
embedding para todas las palabras de la review
'''

model = Sequential([
    vectorize_layer,                                            # 100 [1, 3, 4, 4, 90, ...]
    Embedding(vocab_size, embedding_dim, name="embedding"),     # 10.000 x 16 --> [[], [], [] ...] 100x16
    GlobalAveragePooling1D(),                                   # [] 16
    Dense(16, activation='relu'),
    Dense(1, activation="sigmoid")                              # originalmente no tiene activacion
])

### Ejercicio 10

Entrena el modelo usando un optimizador "Adam" y la función de perdida "BinaryCrossEntropy". Evalua contra test. Prueba con 15 épocas (es muy pesado el entrenamiento).

In [21]:
# https://stackoverflow.com/questions/61233425/what-should-i-use-as-target-vector-when-i-use-binarycrossentropyfrom-logits-tru
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [22]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15
    )

Epoch 1/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 654ms/step - accuracy: 0.5702 - loss: 0.6922 - val_accuracy: 0.6370 - val_loss: 0.6875
Epoch 2/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.6719 - loss: 0.6852 - val_accuracy: 0.6790 - val_loss: 0.6769
Epoch 3/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.6949 - loss: 0.6733 - val_accuracy: 0.7038 - val_loss: 0.6607
Epoch 4/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.7168 - loss: 0.6546 - val_accuracy: 0.7330 - val_loss: 0.6379
Epoch 5/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.7426 - loss: 0.6289 - val_accuracy: 0.7514 - val_loss: 0.6094
Epoch 6/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.7629 - loss: 0.5971 - val_accuracy: 0.7674 - val_loss: 0.5773
Epoch 7/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.7828 - loss: 0.5615 - val_accuracy: 0.7818 - val_loss: 0.5446
Epoch 8/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.7969 - loss: 0.5255 - val_accuracy: 0.7938 -

In [23]:
test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    dataset_dir + "/test/",
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)


Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [24]:
model.evaluate(test_ds)

20/20 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.8059 - loss: 0.4230


[0.4163852632045746, 0.8100000023841858]

### Bonus: Masking

Al limitar el tamaño de secuencia a 100, el modelo está intencionadamente evitando problemas de excesivo padding (las reviews seguramente son de más de 100 palabras), pero...  
El coste es perder capacidad, porque a las 100 palabras ya toma una decisión.  
Si quisieramos que leyese más de 100 palabras o todas, seguramente tendríamos secuencias con mucho padding, y para evitar el problema de este, usaríamos máscaras, con sólo cambiar esto (en la versión fácil)

In [25]:
model = Sequential([
  vectorize_layer,              # 100 [1, 3, 4, 4, 90, ...]
  Embedding(vocab_size, 
            embedding_dim, 
            name="embedding", 
            mask_zero=True),    # la capa  Embedding admite mask, le estamos diciendo que si ve un zero lo ignore
  GlobalAveragePooling1D(),     # [] 16
  Dense(16, activation='relu'), 
  Dense(1)                      # originalmente no tiene activacion
])

Además la capa de Embedding le pasa la mascara a la siguiente capa (en este caso la GlobalAveragePooling1D) a través del argumento mask. Si la capa tiene ese argumento recibe la máscar y la trata como tenga programado.  
En este caso GlobalAveragePooling1D ignora el elemento enmascarado y no lo incluye en la media.   

Una capa recurrente si recibe una máscara ignora los elementos a 0 y para ese "time_step" para ese valor repite el último valor de hidden_state válido

In [26]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

model.fit(train_ds, validation_data=val_ds, epochs=15)

model.evaluate(test_ds)

Epoch 1/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 80ms/step - accuracy: 0.5037 - loss: 2.8781 - val_accuracy: 0.4886 - val_loss: 1.2351
Epoch 2/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.5037 - loss: 1.1370 - val_accuracy: 0.4886 - val_loss: 1.0149
Epoch 3/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.5037 - loss: 0.9578 - val_accuracy: 0.4886 - val_loss: 0.8947
Epoch 4/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.5037 - loss: 0.8540 - val_accuracy: 0.4886 - val_loss: 0.8126
Epoch 5/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.5037 - loss: 0.7828 - val_accuracy: 0.4886 - val_loss: 0.7555
Epoch 6/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.5039 - loss: 0.7342 - val_accuracy: 0.4890 - val_loss: 0.7173
Epoch 7/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - accuracy: 0.5063 - loss: 0.7022 - val_accuracy: 0.5008 - val_loss: 0.6924
Epoch 8/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.5318 - loss: 0.6813 - val_accuracy: 0.5520 - v

[0.589163064956665, 0.7726500034332275]

**¿Y si el valor a enmascarar no es cero?** Podemos usar...

```python
import tensorflow as tf
tensorflow.keras.layers.Masking(mask_value=0.0, **kwargs)
```

mask_value es el valor que queremos "saltarnos" o "enmascarar"